In [1]:
import sys
import os

%load_ext autoreload
%autoreload 2

sys.path.append('../')
sys.path.insert(0, os.path.dirname(os.getcwd()) + "/../")

print(sys.path)

['/home/tiendat/transformer-entropy-ids/../', '/home/tiendat/transformer-entropy-ids/notebooks', '/home/tiendat/miniconda3/envs/torchtf/lib/python39.zip', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [2]:
import pandas as pd
# import vaex
import numpy as np
import glob
import dask.dataframe as dd
import json
from sklearn.model_selection import train_test_split
import math
import csv
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, confusion_matrix
import time
import _warnings
import tensorflow as tf
from tqdm import tqdm
import swifter
import argparse
import helper_functions
from importlib import reload

2023-09-13 13:34:40.233248: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-13 13:34:40.285464: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 13:34:41.037747: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import t

## Get attack

In [3]:
def get_all_data(attack_dict):
    df_aggregation = []
    
    for attack_name, metadata in attack_dict.items():    
        if "accelerator" not in attack_name and "metadata" not in attack_name:
            file_name = '/home/tiendat/transformer-entropy-ids/road/attacks/{}.log'.format(attack_name)
            df_attack = helper_functions.make_can_df(file_name)
            df_attack = helper_functions.add_time_diff_per_aid_col(df_attack)
            df_aggregation.append(df_attack)
            print(f"Finish preprocess {file_name}")
    return df_aggregation

def get_time_interval(attack_dict):
    attack_metadata = []
    
    for attack_name, metadata in attack_dict.items():    
        if "accelerator" not in attack_name and "metadata" not in attack_name:
            print(f"Finish get time interval of {attack_name}")
            attack_metadata.append([tuple(attack_dict[attack_name]["injection_interval"])])
    return attack_metadata

def mark_label(df_aggregation, attack_metadata, attack_dict):
    count = 0
    for attack_name, metadata in attack_dict.items():    
        if "accelerator" not in attack_name and "metadata" not in attack_name:
            print(f"Index {count}: {attack_name} --- {attack_dict[attack_name]['injection_id']}")
            if attack_dict[attack_name]["injection_id"] != "XXX":
                df_aggregation[count] = helper_functions.add_actual_attack_col(df_aggregation[count], attack_metadata[count], int(attack_dict[attack_name]["injection_id"], 16), attack_dict[attack_name]["injection_data_str"], attack_name)
                print(len(df_aggregation[count][df_aggregation[count]['label'] == True]['label']))
            else:
                df_aggregation[count] = helper_functions.add_actual_attack_col(df_aggregation[count], attack_metadata[count], "XXX", attack_dict[attack_name]["injection_data_str"], attack_name)
                print(len(df_aggregation[count][df_aggregation[count]['label'] == True]['label']))
            count += 1
    return df_aggregation

def filter_attack(arr, keyword):
    sub_array = []
    for item in arr:
        if keyword in item:
            sub_array.append(item)
    return sub_array

In [4]:
with open("/home/tiendat/transformer-entropy-ids/road/attacks/capture_metadata.json", "r") as read_file:
    attack_dict = json.load(read_file)

# display(attack_dict)

df_attack = get_all_data(attack_dict)
attack_metadata = get_time_interval(attack_dict)
df_attack = mark_label(df_attack, attack_metadata, attack_dict)

Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_1.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_1_masquerade.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_2.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_2_masquerade.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_3.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/correlated_signal_attack_3_masquerade.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/fuzzing_attack_1.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/fuzzing_attack_2.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/fuzzing_attack_3.log
Finish preprocess /home/tiendat/transformer-entropy-ids/road/attacks/max_engine_coolant_temp_atta

## Get normal

In [5]:
df_normal = []

for file_name in os.listdir("/home/tiendat/transformer-entropy-ids/road/ambient"):
    if "metadata" not in file_name:
        print(file_name)
        file_name = '/home/tiendat/transformer-entropy-ids/road/ambient/' + file_name
        df = helper_functions.make_can_df(file_name)
        df = helper_functions.add_time_diff_per_aid_col(df)
        df['label'] = [False] * df.shape[0]
        print(df.shape)
        df_normal.append(df)

ambient_dyno_exercise_all_bits.log
(4135914, 6)
ambient_highway_street_driving_long.log
(8264830, 6)
ambient_dyno_drive_extended_short.log
(741466, 6)
ambient_dyno_drive_basic_short.log
(996375, 6)
ambient_dyno_drive_benign_anomaly.log
(720822, 6)
ambient_highway_street_driving_diagnostics.log
(1053702, 6)
ambient_dyno_idle_radio_infotainment.log
(1472991, 6)
ambient_dyno_reverse.log
(115180, 6)
ambient_dyno_drive_radio_infotainment.log
(873911, 6)
ambient_dyno_drive_extended_long.log
(1334942, 6)
ambient_dyno_drive_basic_long.log
(2802326, 6)
ambient_dyno_drive_winter.log
(106832, 6)


## Extraction

In [6]:
df_attack_1 = pd.concat(df_attack)
df_normal_1 = pd.concat(df_normal)

df_all = []

df_all.append(df_attack_1)
df_all.append(df_normal_1)

df_all = pd.concat(df_all)
df_all.shape

(25677443, 6)

In [7]:
df_all_normal = df_all[df_all['label'] == False]
df_all_attack = df_all[df_all['label'] == True]
df_all_normal.shape, df_all_attack.shape

((25614610, 6), (62833, 6))

In [8]:
df_normal_1 = pd.concat(df_normal)
display(df_normal_1.shape)
# 22619291
display(df_normal_1[df_normal_1['label'] == True].shape)

(22619291, 6)

(0, 6)

In [9]:
# Initialize an empty list to store the new DataFrames
sub_dfs_normal = []
sub_dfs_29 = []
stride = 15

for subd in df_normal:
    # Loop through the original DataFrame in steps of 15 rows
    subd = subd.sort_values(by=['time'], ascending=True)
    print(len(subd))
    for i in range(0, len(subd), stride):
        if len(subd) - i < stride:
            print(len(subd)-i)
            sub_df = subd.iloc[i:len(subd)-1]
        else:
            sub_df = subd.iloc[i:i+stride]
        sub_dfs_normal.append(sub_df)

4135914
9
8264830
10
741466
1
996375
720822
12
1053702
12
1472991
6
115180
10
873911
11
1334942
2
2802326
11
106832
2


In [10]:
from collections import defaultdict
import numpy as np
import time

max_sub_df = []
min_sub_df = []

def add_entropy_value(df, h_x, h_y, h_y_gv_x, mi, rule_entropy_dict):
    aid = df['aid'].iloc[0]
    if aid not in rule_entropy_dict:
        if aid == 1760 or aid == 208:
            print("=====================================")
            print(f"Saw {aid} for the first time with mi = {mi}")
            print(f"Entropy of X: {h_x}")
            print(f"Entropy of Y: {h_y}")
            print(f"Entropy of Y given X: {h_y_gv_x}")
            
        rule_entropy_dict[aid] = { 'max_mi': mi, 'min_mi': mi, 
                                  'max_en_x': h_x, 'min_en_x': h_x, 
                                  'max_en_y': h_y, 'min_en_y': h_y, 
                                  'max_en_y_x': h_y_gv_x, 'min_en_y_x': h_y_gv_x}
    else:
        if (mi > rule_entropy_dict[aid]['max_mi'] 
            or mi < rule_entropy_dict[aid]['min_mi']
            or h_x > rule_entropy_dict[aid]['max_en_x'] 
            or h_x < rule_entropy_dict[aid]['min_en_x']
            or h_y > rule_entropy_dict[aid]['max_en_y'] 
            or h_y < rule_entropy_dict[aid]['min_en_y']
            or h_y_gv_x > rule_entropy_dict[aid]['max_en_y_x']
            or h_y_gv_x < rule_entropy_dict[aid]['min_en_y_x']) and (aid == 1760 or aid == 208):
            print("=====================================")
            print(f"Saw {aid} for the another time with mi = {mi}")
            print(f"Entropy of X: {h_x}")
            print(f"Entropy of Y: {h_y}")
            print(f"Entropy of Y given X: {h_y_gv_x}")
            max_sub_df.append(df)
        # Check mutual information
        max_mi = max(rule_entropy_dict[aid]['max_mi'], mi)
        min_mi = min(rule_entropy_dict[aid]['min_mi'], mi)
        
        # Check entropy of X
        max_en_x = max(rule_entropy_dict[aid]['max_en_x'], h_x)
        min_en_x = min(rule_entropy_dict[aid]['min_en_x'], h_x)
        
        # Check entropy of Y
        max_en_y = max(rule_entropy_dict[aid]['max_en_y'], h_y)
        min_en_y = min(rule_entropy_dict[aid]['min_en_y'], h_y)
        
        # Check entropy of Y given X
        max_en_y_x = max(rule_entropy_dict[aid]['max_en_y_x'], h_y_gv_x)
        min_en_y_x = min(rule_entropy_dict[aid]['min_en_y_x'], h_y_gv_x)
        
        rule_entropy_dict[aid]['max_en_x'] = max_en_x
        rule_entropy_dict[aid]['min_en_x'] = min_en_x
        
        rule_entropy_dict[aid]['max_en_y'] = max_en_y
        rule_entropy_dict[aid]['min_en_y'] = min_en_y
        
        rule_entropy_dict[aid]['max_en_y_x'] = max_en_y_x
        rule_entropy_dict[aid]['min_en_y_x'] = min_en_y_x
        
        rule_entropy_dict[aid]['max_mi'] = max_mi
        rule_entropy_dict[aid]['min_mi'] = min_mi
        return rule_entropy_dict

def create_threshold_dict(df, h_x, h_y, h_y_gv_x, mi, rule_entropy_dict, count):
    if 'max_mi' not in rule_entropy_dict:
        rule_entropy_dict = { 'max_mi': mi, 'min_mi': mi, 
                                  'max_en_x': h_x, 'min_en_x': h_x, 
                                  'max_en_y': h_y, 'min_en_y': h_y, 
                                  'max_en_y_x': h_y_gv_x, 'min_en_y_x': h_y_gv_x}
        with open("/home/tiendat/transformer-entropy-ids/extract.txt", "a+") as wrtie_file:
            wrtie_file.write(f"\nFirst==========================================\n")
            wrtie_file.write(f"Mutual information: {mi}\n")
            wrtie_file.write(f"Entropy of X: {h_x}\n")
            wrtie_file.write(f"Entropy of Y: {h_y}\n")
            wrtie_file.write(f"Entropy of Y given X: {h_y_gv_x}\n")
    else:
        if (mi > rule_entropy_dict['max_mi'] 
            or mi < rule_entropy_dict['min_mi']
            or h_x > rule_entropy_dict['max_en_x'] 
            or h_x < rule_entropy_dict['min_en_x']
            or h_y > rule_entropy_dict['max_en_y'] 
            or h_y < rule_entropy_dict['min_en_y']
            or h_y_gv_x > rule_entropy_dict['max_en_y_x']
            or h_y_gv_x < rule_entropy_dict['min_en_y_x']):
            max_sub_df.append(df)
            with open("/home/tiendat/transformer-entropy-ids/extract.txt", "a+") as wrtie_file:
                wrtie_file.write(f"\nUpdated==========================================\n")
                wrtie_file.write(f"Mutual information: {mi}\n")
                wrtie_file.write(f"Entropy of X: {h_x}\n")
                wrtie_file.write(f"Entropy of Y: {h_y}\n")
                wrtie_file.write(f"Entropy of Y given X: {h_y_gv_x}\n")
        if (mi == 0 or h_x == 0 or h_y == 0):
            print(df)
            
        # Check mutual information
        max_mi = max(rule_entropy_dict['max_mi'], mi)
        min_mi = min(rule_entropy_dict['min_mi'], mi)
        
        # Check entropy of X
        max_en_x = max(rule_entropy_dict['max_en_x'], h_x)
        min_en_x = min(rule_entropy_dict['min_en_x'], h_x)
        
        # Check entropy of Y
        max_en_y = max(rule_entropy_dict['max_en_y'], h_y)
        min_en_y = min(rule_entropy_dict['min_en_y'], h_y)
        
        # Check entropy of Y given X
        max_en_y_x = max(rule_entropy_dict['max_en_y_x'], h_y_gv_x)
        min_en_y_x = min(rule_entropy_dict['min_en_y_x'], h_y_gv_x)
        
        rule_entropy_dict['max_en_x'] = max_en_x
        rule_entropy_dict['min_en_x'] = min_en_x
        
        rule_entropy_dict['max_en_y'] = max_en_y
        rule_entropy_dict['min_en_y'] = min_en_y
        
        rule_entropy_dict['max_en_y_x'] = max_en_y_x
        rule_entropy_dict['min_en_y_x'] = min_en_y_x
        
        rule_entropy_dict['max_mi'] = max_mi
        rule_entropy_dict['min_mi'] = min_mi
        count += 1
    return rule_entropy_dict, count

def calculate_entropy_ver2(X):
    p_x = defaultdict(float)
    n = len(X)
    
    for x in X:
        p_x[x] += 1 / n
    
    H_X = 0
    for x in p_x:
        H_X += -p_x[x] * np.log2(p_x[x])
    return H_X

def calculate_conditional_entropy_ver2(X, Y):
    p_x = defaultdict(float)
    p_y_given_x = defaultdict(lambda: defaultdict(float))
    
    n = len(X)
    
    for x, y in zip(X, Y):
        p_x[x] += 1 / n
        p_y_given_x[x][y] += 1
    
    for x in p_y_given_x:
        total = sum(p_y_given_x[x].values())
        for y in p_y_given_x[x]:
            p_y_given_x[x][y] /= total
    
    H_Y_given_X = 0
    for x in p_x:
        for y in p_y_given_x[x]:
            H_Y_given_X += p_x[x] * p_y_given_x[x][y] * -np.log2(p_y_given_x[x][y])
    return H_Y_given_X

def calculate_mutual_information(df):
    X = df['aid']
    Y = df['data']
    H_X = calculate_entropy_ver2(X)
    H_Y = calculate_entropy_ver2(Y)
    H_Y_given_X = calculate_conditional_entropy_ver2(X, Y)
    I_X_Y = H_Y - H_Y_given_X
    return H_X, H_Y, H_Y_given_X, I_X_Y

pd.set_option('display.max_rows', None)
# for subd in sub_dfs_normal:
#     subd.groupby('aid').apply(calculate_mutual_information)

In [11]:
rule_entropy_dict = {}
count = 0

for subd in sub_dfs_normal:
    if len(subd) < 15:
        print(subd)
        continue
    H_X, H_Y, H_Y_given_X, I_X_Y = calculate_mutual_information(subd)
    rule_entropy_dict, count = create_threshold_dict(subd, H_X, H_Y, H_Y_given_X, I_X_Y, rule_entropy_dict, count)
    # subd.groupby('aid').apply(calculate_mutual_information)
print(rule_entropy_dict, count)

                time   aid              data      ori_time  time_diffs  label
4419411  2172.794504  1505  893FE00802000080  1.050002e+09    0.010266  False
4419412  2172.794505   354  00080003EA11F4C6  1.050002e+09    0.020010  False
4419413  2172.794506   167  0090FA009105D0A0  1.050002e+09    0.010265  False
4419414  2172.795509   208  0273046400000000  1.050002e+09    0.009189  False
4419415  2172.795510    51  0007D8000C4007D0  1.050002e+09    0.009188  False
4419416  2172.795511  1399  00000000000000AE  1.050002e+09    0.042105  False
4419417  2172.796796    14  205356020808753A  1.050002e+09    0.009458  False
4419418  2172.799462  1760  0000000000000000  1.050002e+09    0.007976  False
                time   aid              data      ori_time  time_diffs  label
8821903  3764.314535   208  7A740464ED0000D1  1.110004e+09    0.011753  False
8821904  3764.314536    51  000768000FC007D0  1.110004e+09    0.010731  False
8821905  3764.314537   628  00F20FED107C8EB9  1.110004e+09    0.

In [ ]:
max_sub_df

In [37]:
max_sub_df[6]['aid'].tolist()

[263, 186, 263, 813, 263, 186, 263, 263, 186, 1307, 813, 60, 519, 1225, 470]

In [39]:
# X = [6, 6, 37, 208, ]
# Y = ['0273046400000000', '0000000000000000', '0000000000000000', '3A770C60F5000000']
X = [263, 186, 263, 813, 263, 186, 263, 813, 263, 186, 263, 263, 186, 813, 60]
Y = ['0000000000000000',
 '04A7F484000606A0',
 '0000000000000000',
 '8000042758010000',
 '0000000000000000',
 '04A7F4C4000606A0',
 '0000000000000000',
 '8000042758010000',
 '0000000000000000',
 '04A7F4C4000606A0',
 '0000000000000000',
 '0000000000000000',
 '04A7F4C4000606A0',
 '8000042758010000',
 '000004002E000000']
# X = ['0E','4A','0E','4B']
# Y = ['0E','5A','0E','53']
H_X = calculate_entropy_ver2(X)
H_Y = calculate_entropy_ver2(Y)
H_Y_given_X = calculate_conditional_entropy_ver2(X, Y)

I_X_Y = H_Y - H_Y_given_X

display(H_X)
display(H_Y)
display(H_Y_given_X)
display(I_X_Y)

1.7464657985040721

1.9628066316931743

0.2163408331891021

1.7464657985040721

In [10]:
max_sub_df, mi_val

([                time  aid              data      ori_time  time_diffs  label
  4419414  2172.795509  208  0273046400000000  1.050002e+09    0.009189  False
  4419391  2172.786320  208  0273046400000000  1.050002e+09    0.010825  False,
                  time   aid              data      ori_time  time_diffs  label
  4419418  2172.799462  1760  0000000000000000  1.050002e+09    0.007976  False
  4419405  2172.791486  1760  0000000000000000  1.050002e+09    0.011934  False,
                  time  aid              data      ori_time  time_diffs  label
  4414323  2169.946025  208  0A730464FB000000  1.050002e+09    0.008013  False
  4414306  2169.938012  208  02730464FC000000  1.050002e+09    0.012007  False,
                  time   aid              data      ori_time  time_diffs  label
  3847531  1932.872471  1760  0054005A00580053  1.050002e+09    0.008245  False
  3847513  1932.864226  1760  005A005F005E0057  1.050002e+09    0.012263  False,
                  time  aid              d

In [12]:

def process_payload(payload):
    """Convert a payload into a list of 8 elements, each of 2 bytes."""
    return [[payload[i:i+2]] for i in range(0, len(payload), 2)]

def update_pattern_list(pattern_list, new_payload):
    """Update the pattern list with new 2-byte values from a new payload."""
    new_values = process_payload(new_payload)
    for i in range(8):
        # If the value is not already in the list for that index, append it
        if new_values[i][0] not in pattern_list[i]:
            pattern_list[i].append(new_values[i][0])

def get_patterns_from_df(df, pattern_dict):
    """Get a list of patterns from a DataFrame."""
    aid = df['aid'].iloc[0]
    pattern_list = [[] for _ in range(8)]
    
    start = time.time()
    for payload in df['data']:
        update_pattern_list(pattern_list, payload)
    
    print(f"Time taken: {time.time() - start} seconds for aid {aid}")
    pattern_dict[str(aid)] = pattern_list
    return pattern_list

def check_single_payload_matching(payload, pattern_list):
    """Check if a single payload matches a pattern list."""
    new_values = process_payload(payload)
    for i in range(8):
        if new_values[i][0] not in pattern_list[i]:
            return True
    return False

def check_payload_matching(df, pattern_dict):
    """Check if a payload matches a pattern list."""
    aid = df['aid'].iloc[0]
    detected_label = df['data'].apply(lambda x: check_single_payload_matching(x, pattern_dict[str(aid)]))
    return detected_label

In [13]:
pattern_dict = {}
df_all_normal.groupby('aid').apply(get_patterns_from_df, pattern_dict)

Time taken: 0.025541305541992188 seconds for aid 6
Time taken: 3.807551860809326 seconds for aid 14
Time taken: 0.04278993606567383 seconds for aid 37
Time taken: 6.030519485473633 seconds for aid 51
Time taken: 0.05768275260925293 seconds for aid 58
Time taken: 0.28849220275878906 seconds for aid 60
Time taken: 0.4703357219696045 seconds for aid 61
Time taken: 0.026808738708496094 seconds for aid 65
Time taken: 0.10098838806152344 seconds for aid 117
Time taken: 3.4111287593841553 seconds for aid 167
Time taken: 1.2008259296417236 seconds for aid 186
Time taken: 1.234982967376709 seconds for aid 192
Time taken: 0.040313005447387695 seconds for aid 204
Time taken: 6.66971755027771 seconds for aid 208
Time taken: 0.25423622131347656 seconds for aid 215
Time taken: 0.0690009593963623 seconds for aid 241
Time taken: 0.24387860298156738 seconds for aid 244
Time taken: 0.02564072608947754 seconds for aid 248
Time taken: 0.026124954223632812 seconds for aid 253
Time taken: 1.3011114597320557

aid
6       [[08, 00], [00], [00], [64, 00], [00], [00], [...
14      [[20], [52, 53, 51, 54, 55, 56, 50, 67, 4F, 4E...
37      [[00], [00], [02, 0A, 0B, 0C, 0D, 0E, 0F, 10, ...
51      [[00, 08, 09, 0B, 0D, 0E, 0F, 10, 11, 12, 13, ...
58      [[00], [00], [00], [00], [00], [1D, 1A, 17, 16...
60      [[00], [00], [04, 00], [00], [2E, 2F, 2D, 2B, ...
61      [[00, 07, 04, 03, 02, 01, 05, 06], [00, 01, D0...
65      [[80, 81], [22, A2, 42], [00], [20], [01, 00],...
117     [[0F, 0E, 00, 0D, 0C, 0B, 08, 09, 0A], [32, 8E...
167     [[00, 10, 20, 30], [10, 11, 70, 71, 51, 50, 90...
186     [[06, 05, 04, 03], [B8, A8, 98, C8, 67, 17, C7...
192     [[60, 20, 00, 40, A0], [00], [00, 10], [00], [...
204     [[43], [41, 40, 31, 39, 38], [04, 05, 06, 07, ...
208     [[52, 5A, 62, 6A, 72, 7A, 02, 0A, 12, 1A, 22, ...
215     [[00], [00, 01, 03, 02], [02, 01, 1A, 52, 6A, ...
241     [[3E, 64, 42, 3B, 3A, 3C, 3D, 43, 44, 48, 4A, ...
244     [[9A, 9F, 95, 96, 99], [0C], [01], [02], [04, ...
248     [[

In [46]:
pattern_dict[6]['maxtime'] = 12

KeyError: 6

In [101]:
detected_label = df_all_attack.groupby('aid').apply(check_payload_matching, pattern_dict)

In [104]:
value_counts = detected_label.value_counts()
value_counts

data
True     34516
False    28317
Name: count, dtype: int64

In [ ]:
with open('/home/tiendat/transformer-entropy-ids/pattern_dict.json', 'w+') as fp:
    json.dump(pattern_dict, fp)
    
for key, value in pattern_dict.items():
    for i in range(len(value)):
        print(f"Length of pattern list for aid {key} at index {i}: {len(value[i])}")

In [ ]:
# FOR ATTACK DATAFRAMEs
sub_dfs_attack = []
stride = 15

for subd in df_attack:
    # Loop through the original DataFrame in steps of 15 rows
    subd = subd.sort_values(by=['time'], ascending=False)
    print(len(subd))
    for i in range(0, len(subd), stride):
        if len(subd) - i < stride:
            print(len(subd)-i)
            sub_df = subd.iloc[i:len(subd)-1]
        else:
            sub_df = subd.iloc[i:i+stride]
        sub_dfs_attack.append(sub_df)

In [253]:
import time
from collections import defaultdict
import numpy as np

def calculate_entropy_ver2(X):
    p_x = defaultdict(float)
    n = len(X)
    
    for x in X:
        p_x[x] += 1 / n
    
    H_X = 0
    for x in p_x:
        H_X += -p_x[x] * np.log2(p_x[x])
    return H_X

def calculate_conditional_entropy_ver2(X, Y):
    p_x = defaultdict(float)
    p_y_given_x = defaultdict(lambda: defaultdict(float))
    
    n = len(X)
    
    for x, y in zip(X, Y):
        p_x[x] += 1 / n
        p_y_given_x[x][y] += 1
    
    for x in p_y_given_x:
        total = sum(p_y_given_x[x].values())
        for y in p_y_given_x[x]:
            p_y_given_x[x][y] /= total
    
    H_Y_given_X = 0
    for x in p_x:
        for y in p_y_given_x[x]:
            H_Y_given_X += p_x[x] * p_y_given_x[x][y] * -np.log2(p_y_given_x[x][y])
    
    return H_Y_given_X

def check_mutual_information(df, rule_mi_dict):
    X = df['aid']
    Y = df['data']
    H_X = calculate_entropy(X)
    H_Y = calculate_entropy(Y)
    H_Y_given_X = calculate_conditional_entropy(X, Y)
    
    I_X_Y = H_Y - H_Y_given_X
    # if I_X_Y > rule_mi_dict[X.iloc[0]]['max_mi']:
    #     return True
    return I_X_Y
    # return I_X_Y

total = 0
attack_real = 0
attack_predict = 0
subd_attacked = []
mi_attacked = []

for subd in sub_dfs:
    start = time.time()
    total += len(subd)    
    predict_result = subd.groupby('aid').apply(check_mutual_information, rule_mi_dict)
    
    if True in subd['label'].unique():
        attack_real += 1
        subd_attacked.append(subd)
        mi_attacked.append(predict_result)
    
    if predict_result.any().any():
        attack_predict += 1
    
print(f"Total: {total}")
print(f"Attack real: {attack_real}")
print(f"Attack predict: {attack_predict}")
print(f"Accuracy: {attack_predict/total}")
print(f"Recall: {attack_predict/attack_real}")
print(f"Precision: {attack_predict/(attack_predict + (total - attack_real))}")
print(f"F1 score: {2 * (attack_predict/(attack_predict + (total - attack_real))) * (attack_predict/attack_real) / ((attack_predict/(attack_predict + (total - attack_real))) + (attack_predict/attack_real))}")
display(subd_attacked, mi_attacked)

Total: 3058125
Attack real: 59821
Attack predict: 0
Accuracy: 0.0
Recall: 0.0
Precision: 0.0


ZeroDivisionError: float division by zero

In [ ]:
display(mi_attacked)

In [156]:
df_normal_1 = pd.concat(df_normal)
len(sub_dfs), len(df_normal_1)/15

(203890, 1507952.7333333334)

In [ ]:
# df_temp = df_attack.loc[(df_attack['label'] == True) & (df_attack['aid'] == 1255) & (df_attack['time_diffs'] < 0.2)]

sum_time = df_all.groupby(['aid', 'label']).agg({'data': ['count']})
pd.set_option('display.max_rows', None)
display(sum_time)

In [7]:
df_all_normal = df_all[df_all['label'] == False]
display(df_all_normal)

,time,aid,data,ori_time,time_diffs,label
3873,1.618163,6,0800006400000000,1.030000e+09,0.999845,False
6266,2.618064,6,0800006400000000,1.030000e+09,0.999901,False
8655,3.617806,6,0800006400000000,1.030000e+09,0.999742,False
11049,4.617810,6,0800006400000000,1.030000e+09,1.000004,False
13441,5.618164,6,0800006400000000,1.030000e+09,1.000354,False
...,...,...,...,...,...,...
113058,47.280774,1788,0000077B90C2C000,1.050000e+09,0.099978,False
113297,47.381792,1788,0000077B10C2B800,1.050000e+09,0.101018,False
113533,47.481730,1788,0000077990C2B800,1.050000e+09,0.099938,False
113783,47.581725,1788,0000077AD0C2B800,1.050000e+09,0.099995,False


In [8]:
df_sorted = df_all_normal.sort_values(by=['time'])
display(df_sorted)

,time,aid,data,ori_time,time_diffs,label
16,0.006927,167,0010FA27012B90A0,1.030000e+09,0.006927,False
18,0.008660,1760,015D015D01560159,1.000000e+09,0.007658,False
26,0.008952,293,90002DDF20383F60,1.200000e+09,0.008952,False
26,0.008952,293,90002DDF20383F60,1.170000e+09,0.008952,False
25,0.009104,208,6A710460EF000000,1.190000e+09,0.009104,False
...,...,...,...,...,...,...
8821908,3764.315521,1076,030AA0A8D194B6B0,1.110004e+09,0.019963,False
8821909,3764.315522,1760,0000000000000000,1.110004e+09,0.013771,False
8821910,3764.315522,192,6000000000000000,1.110004e+09,0.018924,False
8821911,3764.316567,1644,0080100020000020,1.110004e+09,0.099955,False


In [7]:
df_unique = df_all.drop_duplicates(subset=['aid', 'data'])
display(df_unique)

,time,aid,data,ori_time,time_diffs,label
3873,1.618163,6,0800006400000000,1.030000e+09,0.999845,False
33,0.014802,14,205296020809767A,1.030000e+09,0.009998,False
54,0.024708,14,2052960208097674,1.030000e+09,0.009906,False
128,0.054923,14,205296020809766E,1.030000e+09,0.010121,False
243,0.104714,14,2052960208097680,1.030000e+09,0.009938,False
...,...,...,...,...,...,...
112587,47.080794,1788,00000779F0C2C000,1.050000e+09,0.099962,False
113058,47.280774,1788,0000077B90C2C000,1.050000e+09,0.099978,False
113297,47.381792,1788,0000077B10C2B800,1.050000e+09,0.101018,False
113533,47.481730,1788,0000077990C2B800,1.050000e+09,0.099938,False


In [ ]:
data_unique = df_unique.groupby(['aid', 'label']).agg({'data': ['count']})
pd.set_option('display.max_rows', None)
display(data_unique)

In [44]:
pd.set_option('display.max_rows', 20)
display(df_all)

,time,aid,data,ori_time,time_diffs,label
3873,1.618163,6,0800006400000000,1.030000e+09,0.999845,False
6266,2.618064,6,0800006400000000,1.030000e+09,0.999901,False
8655,3.617806,6,0800006400000000,1.030000e+09,0.999742,False
11049,4.617810,6,0800006400000000,1.030000e+09,1.000004,False
13441,5.618164,6,0800006400000000,1.030000e+09,1.000354,False
...,...,...,...,...,...,...
113058,47.280774,1788,0000077B90C2C000,1.050000e+09,0.099978,False
113297,47.381792,1788,0000077B10C2B800,1.050000e+09,0.101018,False
113533,47.481730,1788,0000077990C2B800,1.050000e+09,0.099938,False
113783,47.581725,1788,0000077AD0C2B800,1.050000e+09,0.099995,False


In [20]:
def split_data_column(row):
    data = row['data']
    chunks = [data[i:i+2] for i in range(0, len(data), 2)]
    return pd.Series(chunks)

In [42]:
df_208 = pd.DataFrame({})
df_208[['data1', 'data2', 'data3', 'data4', 'data5', 'data6', 'data7', 'data8']] = df_unique[(df_unique['aid'] == 1760) & (df_unique['label'] == True)].apply(split_data_column, axis=1)
df_208.to_csv('aid_1760.csv', index=False)

In [43]:
df_unique[(df_unique['aid'] == 1760) & (df_unique['label'] == True)]

,time,aid,data,ori_time,time_diffs,label
21990,9.190831,1760,595945450000FFFF,1.030000e+09,0.003063,True
18760,7.611744,1760,FFFFFFFFFFFFFFFF,1.000000e+09,0.004122,True


In [ ]:
# Extract the amount of data per aid
amount_data_per_aid = df_all_normal.groupby('aid').size()

# Extract max and min time_diffs per aid
max_time_diffs_per_aid = df_all_normal.groupby('aid')['time_diffs'].max()
min_time_diffs_per_aid = df_all_normal.groupby('aid')['time_diffs'].min()

def find_common_pattern(strings):
    # Convert the first string to a NumPy array
    common_pattern = np.array(list(strings[0]))
    
    # Iterate through the rest of the strings
    for string in strings[1:]:
        # Convert the string to a NumPy array
        arr = np.array(list(string))
        
        # Update the common pattern using vectorized operations
        common_pattern[arr != common_pattern] = 'X'
    
    # Convert the NumPy array back to a string
    result = ''.join(common_pattern)

    return result

for aid in amount_data_per_aid.keys():
    print(f"\ncanid {aid} have {amount_data_per_aid[aid]}")
    print(f"max time diff {max_time_diffs_per_aid[aid]} and min time diff {min_time_diffs_per_aid[aid]}")


In [19]:
import copy

# display(pattern_dict)
display(rule_entropy_dict)
rule_all_dict = copy.copy(rule_entropy_dict)

for aid in amount_data_per_aid.keys():
    rule_all_dict[aid] = {"max_time": max_time_diffs_per_aid[aid], "min_time": min_time_diffs_per_aid[aid], "pattern": pattern_dict[str(aid)]}

with open('/home/tiendat/transformer-entropy-ids/rule_all_dict.json', 'w+') as fp:
    json.dump(rule_all_dict, fp)

{'max_mi': 3.906890595608518,
 'min_mi': 1.7464657985040721,
 'max_en_x': 3.906890595608518,
 'min_en_x': 1.7464657985040721,
 'max_en_y': 3.906890595608518,
 'min_en_y': 1.9628066316931743,
 'max_en_y_x': 0.7999999999999999,
 'min_en_y_x': 0.0}

In [54]:
key = [   6,   14,   37,   51,   58,   60,   61,   65,  117,  167,  186,
        192,  204,  208,  215,  241,  244,  248,  253,  263,  293,  300,
        304,  339,  354,  403,  412,  420,  426,  452,  458,  470,  485,
        519,  526,  541,  560,  569,  622,  627,  628,  631,  640,  651,
        661,  663,  675,  676,  683,  692,  695,  705,  722,  727,  737,
        738,  778,  813,  837,  852,  870,  881,  930,  953,  961,  996,
       1031, 1049, 1072, 1076, 1124, 1175, 1176, 1225, 1227, 1255, 1262,
       1277, 1307, 1314, 1331, 1372, 1398, 1399, 1408, 1413, 1455, 1459,
       1505, 1512, 1533, 1560, 1590, 1621, 1628, 1634, 1644, 1649, 1661,
       1668, 1693, 1694, 1751, 1760, 1788]

# parttern = [find_common_pattern(list(df_all.groupby('aid')['data']))]
grouped_data = df_unique.groupby('aid')['data'].apply(list).reset_index()
# grouped_data['pattern'] = grouped_data['data'].apply(lambda x: find_common_pattern(x))
# for aid in key:
#     # print(f"max time diff {max_time_diffs_per_aid[aid]} and min time diff {min_time_diffs_per_aid[aid]} ===========================================")
#     if df_all[df_all['aid'] == aid].shape[0] >= 20 and df_all[df_all['aid'] == aid].shape[0] <= 50:    
#        print(f"{aid}")
#        display(df_all[df_all['aid'] == aid]['data'].reset_index(drop=True))

In [ ]:
test = df_all.groupby('aid')['data'].apply(lambda x: find_common_pattern(x))
display(test)

In [84]:
# Set the option to display all rows and columns
pd.set_option("display.max_rows", 15)
pd.set_option('display.max_colwidth', None)

print(grouped_data['pattern'])

0      0X0000XX00000000
1      20XXX602080XXXXX
2      0000XXX840XXXXX1
3      XXXXXXXXXXXXXXX0
4      0000000000XXXX00
             ...       
100    XXXXXXX0XXXXXXXX
101    XX400XXXXXX0XXXX
102    XXXXXX0XX0XXXXXX
103    XXXXXXXXXXXXXXXX
104    X000XXXXXXXXXXXX
Name: pattern, Length: 105, dtype: object


# Entropy

In [186]:
from collections import defaultdict
import numpy as np
import time

def calculate_entropy_ver2(X):
    p_x = defaultdict(float)
    n = len(X)
    
    for x in X:
        p_x[x] += 1 / n
    
    H_X = 0
    for x in p_x:
        H_X += -p_x[x] * np.log2(p_x[x])
    return H_X

def calculate_conditional_entropy_ver2(X, Y):
    p_x = defaultdict(float)
    p_y_given_x = defaultdict(lambda: defaultdict(float))
    
    n = len(X)
    
    for x, y in zip(X, Y):
        p_x[x] += 1 / n
        p_y_given_x[x][y] += 1
    
    for x in p_y_given_x:
        total = sum(p_y_given_x[x].values())
        for y in p_y_given_x[x]:
            p_y_given_x[x][y] /= total
    
    H_Y_given_X = 0
    for x in p_x:
        for y in p_y_given_x[x]:
            H_Y_given_X += p_x[x] * p_y_given_x[x][y] * -np.log2(p_y_given_x[x][y])
    
    return H_Y_given_X

def calculate_mutual_information(df):
    X = df['aid']
    Y = df['data']
    H_X = calculate_entropy(X)
    H_Y = calculate_entropy(Y)
    H_Y_given_X = calculate_conditional_entropy(X, Y)
    
    I_X_Y = H_Y - H_Y_given_X
    return I_X_Y

In [ ]:
for subd in sub_dfs:
    if subd['aid'].unique()[0] == 1760:
        print(subd['data'].unique())
        print(calculate_entropy_ver2(subd['data'].unique()))
        print(calculate_mutual_information(subd))

In [189]:
for subd in sub_dfs:
    time_start = time.time()
    list_entropy = subd.groupby('aid').apply(calculate_mutual_information)
    print(f"Time: {time.time() - time_start}")
    print(list_entropy)
    aa

Time: 0.002415895462036133
aid
14      0.0
51      0.0
61      0.0
167     0.0
186     0.0
208     0.0
263     0.0
293     0.0
651     0.0
852     0.0
961     0.0
1413    0.0
1505    0.0
1760    0.0
dtype: float64


NameError: name 'aa' is not defined

In [184]:
calculate_mutual_information('03B103AB03A403A4')

1.75

In [124]:
import re
# Define the Regex method
def match_pattern_regex(string, pattern):
    regex_pattern = pattern.replace('X', '.')
    return bool(re.fullmatch(regex_pattern, string))

def extract_X_positions(payload, pattern):
    extracted_chars = [payload[i] for i, char in enumerate(pattern) if char == 'X']
    return ''.join(extracted_chars)

In [188]:
# Re-importing the required libraries and re-running the payload_to_dataframe function
import pandas as pd
# Re-define the function to calculate Mutual Information, as the code state was reset
from collections import Counter
from math import log2

def calculate_mutual_information(df):
    # Calculate the joint probability distribution p(x, y)
    joint_prob = Counter(zip(df['X'], df['Y']))
    total_count = len(df)
    joint_prob = {key: value / total_count for key, value in joint_prob.items()}

    # Calculate the marginal probabilities p(x) and p(y)
    prob_x = Counter(df['X'])
    prob_y = Counter(df['Y'])
    prob_x = {key: value / total_count for key, value in prob_x.items()}
    prob_y = {key: value / total_count for key, value in prob_y.items()}
    
    # Calculate Mutual Information I(X;Y)
    mutual_info = 0
    for (x, y), p_xy in joint_prob.items():
        p_x = prob_x.get(x, 0)
        p_y = prob_y.get(y, 0)
        
        if p_x > 0 and p_y > 0:
            mutual_info += p_xy * log2(p_xy / (p_x * p_y))
    
    return mutual_info


# Function to convert payload into a DataFrame containing columns X and Y
def payload_to_dataframe(payload):
    # Split the payload into X and Y parts
    X_values = [payload[i:i+1] for i in range(0, len(payload)//2, 1)]
    Y_values = [payload[i:i+1] for i in range(len(payload)//2, len(payload), 1)]
    
    # Create a DataFrame
    df = pd.DataFrame({
        'X': X_values,
        'Y': Y_values
    })
    
    return calculate_mutual_information(df)

def calculate_conditional_entropy(string):
    X = string[0:8]
    Y = string[8:16]
    p_x = defaultdict(float)
    p_y_given_x = defaultdict(lambda: defaultdict(float))
    
    n = len(X)
    
    for x, y in zip(X, Y):
        p_x[x] += 1 / n
        p_y_given_x[x][y] += 1
    
    for x in p_y_given_x:
        total = sum(p_y_given_x[x].values())
        for y in p_y_given_x[x]:
            p_y_given_x[x][y] /= total
    
    H_Y_given_X = 0
    for x in p_x:
        for y in p_y_given_x[x]:
            H_Y_given_X += p_x[x] * p_y_given_x[x][y] * -np.log2(p_y_given_x[x][y])
    
    return H_Y_given_X

In [193]:
calculate_mutual_information('595945450000FFFF')

0.5

In [189]:
df_1255_normal = df_unique[df_unique['aid'] == 1760]
# df_1255_attack = df_attack[df_attack['aid'] == 1760]

df_1255_normal['mi'] = df_1255_normal['data'].apply(lambda x: calculate_conditional_entropy(x))
# df_1255_attack['mi'] = df_1255_attack['data'].apply(lambda x: payload_to_dataframe(x))
print(df_1255_normal['mi'].min(), df_1255_normal['mi'].max())

0.0 2.0


/tmp/ipykernel_2894134/2099272133.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1255_normal['mi'] = df_1255_normal['data'].apply(lambda x: calculate_conditional_entropy(x))


In [157]:
display(df_1255_attack[df_1255_attack['data']])

AttributeError: 'Series' object has no attribute 'include'

In [19]:
import pandas as pd
from collections import Counter
from math import log2

# Generate a synthetic dataset for demonstration
# Assuming CanID is an integer < 2000 and Data is a 16-character hexadecimal string
import random

random.seed(42)

# Split the 'Data' column into two separate columns for X and Y
df_sorted['X'] = df_sorted['data'].apply(lambda x: x[:8])
df_sorted['Y'] = df_sorted['data'].apply(lambda x: x[8:])

In [ ]:
import pandas as pd
from collections import Counter
from math import log2

# Function to calculate Conditional Mutual Information for a given DataFrame
def calculate_CMI(df):
    # Calculate the joint probability distribution p(x, y, z)
    joint_prob = Counter(zip(df['X'], df['Y'], df['aid']))
    total_count = len(df)
    joint_prob = {key: value / total_count for key, value in joint_prob.items()}

    # Initialize variables to hold conditional probabilities and sums
    cond_prob_x_given_z = {}
    cond_prob_y_given_z = {}
    cond_prob_xy_given_z = {}
    sum_prob_given_z = Counter(df['aid'])

    # Calculate conditional probabilities
    for (x, y, z), p_xyz in joint_prob.items():
        sum_z = sum_prob_given_z[z] / total_count
        cond_prob_x_given_z[(x, z)] = p_xyz / sum_z if sum_z > 0 else 0
        cond_prob_y_given_z[(y, z)] = p_xyz / sum_z if sum_z > 0 else 0
        cond_prob_xy_given_z[(x, y, z)] = p_xyz / sum_z if sum_z > 0 else 0

    # Calculate Conditional Mutual Information (CMI)
    CMI = 0
    for (x, y, z), p_xyz in joint_prob.items():
        pxz = cond_prob_x_given_z.get((x, z), 0)
        pyz = cond_prob_y_given_z.get((y, z), 0)
        pxyz = cond_prob_xy_given_z.get((x, y, z), 0)
        
        if pxz > 0 and pyz > 0 and pxyz > 0:
            CMI += p_xyz * log2(pxyz / (pxz * pyz))
    
    return CMI

# Calculate CMI for each set of 15 rows in the DataFrame
CMI_values = []
for i in range(0, len(df_sorted), 15):
    if len(df_sorted) - i < 15:
        subset_df = df_sorted.iloc[i:len(df_sorted) - 1]
        CMI_values.append(calculate_CMI(subset_df))
    else:
        subset_df = df_sorted.iloc[i:i+15]
        CMI_values.append(calculate_CMI(subset_df))


In [29]:
CMI_values_attack = [] 
df_attack['X'] = df_attack['data'].apply(lambda x: x[:8])
df_attack['Y'] = df_attack['data'].apply(lambda x: x[8:])
for i in range(0, len(df_attack), 15):
    if len(df_attack) - i < 15:
        subset_df = df_attack.iloc[i:len(df_attack) - 1]
        CMI_values_attack.append(calculate_CMI(subset_df))
    else:
        subset_df = df_sorted.iloc[i:i+15]
        CMI_values_attack.append(calculate_CMI(subset_df))



In [47]:
max(CMI_values_attack)
count = 0
for index, value in enumerate(CMI_values_attack):
    if value > 1.7249535320618146:
        print(index, value)
        count += 1

203876 3.4594316186372978


In [49]:
max(CMI_values)

3.4594316186372978